Mediated schema attributes defined based on our compentency questions.

Dataset columns are is for the Diabetes dataset, after dropping some irrelevent columns that are not used in mediated

In [12]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import jaccard_score

# Input mediated schema and dataset columns as lists
mediated_schema = [
    "Patient_identfication_number", "First_Last_Names", "Age", "Patient_gender", "diabetes",
    "heart_disease_status", "Systolic_BP", "Diaolistic_BP",
    "bmi", "cholesterol", "smoker", "medications"
]
dataset_columns = ['Diabetes_binary', 'HighBP', 'HighChol', 'CholCheck', 'BMI',
                   'Smoker', 'Stroke', 'HeartDiseaseorAttack', 'PhysActivity', 'HvyAlcoholConsump',
                   'Sex', 'Age', 'patient_id', 'name']


Matchers:

First Matcher algorithm implemented is **Jaccard matcher**.

Implementation is done using MultiLabelBinarizer, output result is similar to EditDistence output: **diabetes_jaccard_matrix**

In [13]:
# 2. Jaccard Similarity using sklearn.metrics.jaccard_score

mlb = MultiLabelBinarizer()

#testResults mediated schema
schema_sets = [set(term.lower()) for term in mediated_schema]
dataset_sets = [set(term.lower()) for term in dataset_columns]
all_sets = schema_sets + dataset_sets
binary_matrix = mlb.fit_transform(all_sets)
schema_bin = binary_matrix[:len(mediated_schema)]
dataset_bin = binary_matrix[len(mediated_schema):]

diabetes_jaccard_matrix = pd.DataFrame(index=mediated_schema, columns=dataset_columns)
for i, s_bin in enumerate(schema_bin):
    for j, d_bin in enumerate(dataset_bin):
        sim = jaccard_score(s_bin, d_bin)
        diabetes_jaccard_matrix.iloc[i, j] = round(sim, 3)

diabetes_jaccard_matrix = diabetes_jaccard_matrix.astype(float)
diabetes_jaccard_matrix.to_csv("diabetes_jaccard_matrix.csv")
diabetes_jaccard_matrix


,Diabetes_binary,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,HvyAlcoholConsump,Sex,Age,patient_id,name
Patient_identfication_number,0.529,0.176,0.167,0.167,0.200,0.235,0.235,0.444,0.263,0.350,0.059,0.125,0.533,0.267
First_Last_Names,0.571,0.067,0.133,0.133,0.167,0.308,0.308,0.375,0.250,0.278,0.167,0.167,0.462,0.364
Age,0.167,0.143,0.125,0.125,0.000,0.125,0.125,0.167,0.091,0.071,0.200,1.000,0.222,0.400
Patient_gender,0.615,0.250,0.143,0.067,0.083,0.143,0.231,0.400,0.267,0.158,0.083,0.300,0.800,0.273
diabetes,0.636,0.200,0.083,0.083,0.250,0.182,0.300,0.500,0.333,0.118,0.250,0.250,0.500,0.222
heart_disease_status,0.615,0.154,0.143,0.143,0.083,0.231,0.333,0.615,0.357,0.222,0.182,0.182,0.500,0.167
Systolic_BP,0.400,0.250,0.333,0.231,0.182,0.143,0.231,0.312,0.462,0.375,0.083,0.000,0.286,0.000
Diaolistic_BP,0.467,0.231,0.308,0.214,0.167,0.133,0.214,0.467,0.429,0.353,0.077,0.077,0.462,0.071
bmi,0.167,0.333,0.125,0.000,1.000,0.125,0.000,0.077,0.091,0.071,0.000,0.000,0.100,0.167
cholesterol,0.267,0.083,0.400,0.556,0.000,0.400,0.556,0.583,0.308,0.333,0.222,0.100,0.143,0.091


Second String Matcher is **Soft TF-IDF**.

Implementation is done using cosine similarity lib, output result is similar to EditDistence output: **diabetes_softTfidf_matrix**

In [14]:
# 3. Soft TF-IDF (Cosine Similarity on char n-grams)
def softTfIdf(mediated_schema, csvName):
    vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(1, 3))
    all_terms = mediated_schema + dataset_columns
    tfidf_matrix = vectorizer.fit_transform(all_terms)
    cosine_sim = cosine_similarity(tfidf_matrix[:len(mediated_schema)],
                               tfidf_matrix[len(mediated_schema):])
    soft_tfidf_matrix = pd.DataFrame(
        np.round(cosine_sim, 3),
        index=mediated_schema,
        columns=dataset_columns
    )
    soft_tfidf_matrix.to_csv(csvName)

    return soft_tfidf_matrix


diabetes_softTfidf_matrix = softTfIdf(mediated_schema, "diabetes_softTfidf_matrix.csv")
diabetes_softTfidf_matrix


,Diabetes_binary,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,HvyAlcoholConsump,Sex,Age,patient_id,name
Patient_identfication_number,0.179,0.052,0.038,0.043,0.091,0.075,0.072,0.149,0.115,0.108,0.026,0.055,0.682,0.141
First_Last_Names,0.153,0.009,0.024,0.021,0.045,0.075,0.126,0.141,0.067,0.059,0.057,0.048,0.162,0.378
Age,0.070,0.087,0.073,0.019,0.000,0.029,0.027,0.099,0.020,0.016,0.041,1.000,0.049,0.085
Patient_gender,0.134,0.061,0.034,0.020,0.018,0.083,0.074,0.140,0.081,0.036,0.042,0.199,0.628,0.119
diabetes,0.672,0.047,0.013,0.020,0.085,0.049,0.067,0.156,0.064,0.016,0.070,0.080,0.120,0.064
heart_disease_status,0.170,0.040,0.047,0.063,0.014,0.076,0.125,0.577,0.092,0.060,0.186,0.083,0.176,0.067
Systolic_BP,0.137,0.108,0.090,0.102,0.065,0.048,0.095,0.069,0.163,0.130,0.041,0.000,0.086,0.000
Diaolistic_BP,0.223,0.123,0.103,0.095,0.101,0.031,0.075,0.133,0.118,0.103,0.019,0.022,0.171,0.018
bmi,0.108,0.088,0.024,0.000,1.000,0.061,0.000,0.014,0.036,0.026,0.000,0.000,0.049,0.080
cholesterol,0.105,0.048,0.347,0.373,0.000,0.135,0.202,0.118,0.053,0.218,0.053,0.037,0.041,0.030


Third matcher is the Semantic matcher:

In here we used a pre-trained transformer model: **all-MiniLM-L12-v2**.

**compute_similarity_matrix** method is the entry point to execute the algorithm, simply we compute the embeded for mediated attributes and source attributes.

**source_data, and schema_expectations** are used to help the model execute the schema matching technique.

Similarity output matrix is **diabetes_semantic_matrix**

In [15]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Configuration parameters
MODEL_NAME = "sentence-transformers/all-MiniLM-L12-v2"
NAME_WEIGHT = 0.5
VALUE_WEIGHT = 0.3
EXPECTATION_WEIGHT = 0.2
MAX_SAMPLE_VALUES = 10

# Load the transformer model
model = SentenceTransformer(MODEL_NAME)

def embed_text(text: str) -> np.ndarray:
    return model.encode([str(text)])[0]

def embed_text_list(texts: list[str]) -> np.ndarray:
    if not texts:
        return np.zeros(model.get_sentence_embedding_dimension())
    vectors = model.encode([str(t) for t in texts])
    return np.mean(vectors, axis=0)

def compute_mediated_embedding(col_name: str, expectations: list[str]) -> np.ndarray:
    name_vec = embed_text(col_name)
    if expectations:
        exp_vec = embed_text_list(expectations)
        return NAME_WEIGHT * name_vec + EXPECTATION_WEIGHT * exp_vec
    return name_vec

def compute_source_embedding(col_name: str, values: list[str]) -> np.ndarray:
    name_vec = embed_text(col_name)
    sampled_values = values[:MAX_SAMPLE_VALUES]
    if sampled_values:
        val_vec = embed_text_list(sampled_values)
        return NAME_WEIGHT * name_vec + VALUE_WEIGHT * val_vec
    return name_vec

def compute_similarity_matrix(mediated_schema: list[str], schema_expectations: dict, source_df: pd.DataFrame) -> pd.DataFrame:
    mediated_vectors = {
        col: compute_mediated_embedding(col, schema_expectations.get(col, []))
        for col in mediated_schema
    }

    source_vectors = {
        col: compute_source_embedding(col, source_df[col].astype(str).tolist())
        for col in source_df.columns
    }

    matrix = pd.DataFrame(index=mediated_schema, columns=source_df.columns, dtype=float)
    for m_col, m_vec in mediated_vectors.items():
        for s_col, s_vec in source_vectors.items():
            matrix.loc[m_col, s_col] = cosine_similarity([m_vec], [s_vec])[0][0]
    return matrix



schema_expectations = {
    "Patient_identfication_number": ["508", "819", "453"],
    "First_Last_Names": ["AMANDA MUNOZ", "Xavier Z. Jordan", "E. Lucero"],
    "Age": ["60-64", "50-54"],
    "Patient_gender": ["Female", "Male", "Sex"],
    "diabetes": ["Yes", "No", "Diabetes_binary"],
    "heart_disease_status": ["HeartDiseaseorAttack", "Yes", "No"],
    "Systolic_BP": ["HighBP", "Blood Pressure", "Yes", "No"],
    "Diaolistic_BP": ["N/A"],
    "bmi": ["40.0", "25.0", "28.0"],
    "cholesterol": ["HighChol", "Yes", "No"],
    "smoker": ["Yes", "No", "Smoker"],
    "medications": ["CholCheck", "BPMeds", "No"]
}

source_data = {
    "Diabetes_binary": ["No", "No", "No"],
    "HighBP": ["Yes", "No", "Yes"],
    "HighChol": ["Yes", "No", "Yes"],
    "CholCheck": ["Yes", "No", "Yes"],
    "BMI": [40.0, 25.0, 28.0],
    "Smoker": ["Yes", "Yes", "No"],
    "Stroke": ["No", "No", "No"],
    "HeartDiseaseorAttack": ["No", "No", "No"],
    "PhysActivity": ["No", "Yes", "No"],
    "HvyAlcoholConsump": ["No", "No", "No"],
    "Sex": ["Female", "Female", "Female"],
    "Age": ["60-64", "50-54", "60-64"],
    "patient_id": [508, 819, 453],
    "name": ["AMANDA MUNOZ", "Xavier Z. Jordan", "E. Lucero"]
}

df = pd.DataFrame(source_data)

diabetes_semantic_matrix = compute_similarity_matrix(mediated_schema, schema_expectations, df)
diabetes_semantic_matrix.to_csv("diabetes_semantic_matrix.csv", index=True)
diabetes_semantic_matrix


,Diabetes_binary,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,HvyAlcoholConsump,Sex,Age,patient_id,name
Patient_identfication_number,0.359141,0.183507,0.206759,0.198231,0.161713,0.148549,0.218490,0.243567,0.194660,0.177224,0.197361,0.313373,0.856643,0.200955
First_Last_Names,0.201595,0.194502,0.271767,0.272217,0.110401,0.282803,0.211040,0.243217,0.169078,0.179235,0.301929,0.236880,0.203080,0.624165
Age,0.238885,0.270657,0.315819,0.258657,0.322898,0.288677,0.276510,0.211812,0.203970,0.214280,0.385279,0.993874,0.280071,0.288765
Patient_gender,0.443403,0.301531,0.300565,0.228025,0.233379,0.295399,0.290290,0.330783,0.293495,0.246788,0.659466,0.340199,0.591852,0.310076
diabetes,0.803563,0.355920,0.354140,0.333637,0.227318,0.343627,0.445194,0.423786,0.331066,0.399440,0.331092,0.174044,0.344478,0.269432
heart_disease_status,0.517267,0.383170,0.415781,0.348790,0.242717,0.391071,0.392832,0.678145,0.351337,0.309511,0.270448,0.280267,0.454167,0.255488
Systolic_BP,0.475144,0.501884,0.396061,0.336147,0.265638,0.306833,0.398013,0.506442,0.293346,0.353143,0.194043,0.226294,0.401767,0.250668
Diaolistic_BP,0.549303,0.373801,0.296585,0.287567,0.278971,0.317897,0.354553,0.420375,0.362522,0.366579,0.357100,0.223503,0.404497,0.319807
bmi,0.242472,0.323237,0.160551,0.098944,0.991317,0.169591,0.224685,0.148222,0.214026,0.139636,0.273686,0.290618,0.150883,0.210830
cholesterol,0.401877,0.378792,0.535845,0.562095,0.218648,0.392972,0.405467,0.387505,0.332515,0.324275,0.277928,0.232442,0.283975,0.237131


In [23]:
diabetes_jaccard_matrix.to_latex("diabetes_jaccard_matrix.tex", index=True, float_format="%.3f")
diabetes_softTfidf_matrix.to_latex("diabetes_softTfidf_matrix.tex", index=True, float_format="%.3f")
diabetes_semantic_matrix.to_latex("diabetes_semantic_matrix.tex", index=True, float_format="%.3f")

In next Cell we will implement two combiner algorithm to combine the result of previous 3 matchers to produce a new similarity matrix combines all of the three.

In [16]:
import pandas as pd
import numpy as np

def combine_maximum(*matrices: pd.DataFrame) -> pd.DataFrame:
    """
    Combine multiple similarity matrices by taking the maximum value for each cell.
    """
    stacked = np.stack([m.values for m in matrices], axis=0)
    combined = np.max(stacked, axis=0)
    return pd.DataFrame(combined, index=matrices[0].index, columns=matrices[0].columns)

def combine_weighted_average(matrices: list[pd.DataFrame], weights: list[float]) -> pd.DataFrame:
    """
    Combine multiple similarity matrices using a weighted average.
    Weights must match the number of matrices and sum to 1.
    """
    if len(matrices) != len(weights):
        raise ValueError("Number of weights must match number of matrices")

    stacked = np.stack([m.values for m in matrices], axis=0)
    weighted = np.tensordot(weights, stacked, axes=([0], [0]))

    return pd.DataFrame(weighted, index=matrices[0].index, columns=matrices[0].columns)



# Combine using maximum
max_combined = combine_maximum(diabetes_jaccard_matrix, diabetes_softTfidf_matrix, diabetes_semantic_matrix)
print("Maximum Combined Matrix:")


# Placeholder weights for now — we'll update these later
weights = [0.2, 0.2, 0.6]  # Must sum to 1
weighted_combined = combine_weighted_average([diabetes_jaccard_matrix, diabetes_softTfidf_matrix, diabetes_semantic_matrix], weights)
print("\nWeighted Average Combined Matrix:")


# Save results
max_combined.to_csv("combined_maximum_matrix.csv")
weighted_combined.to_csv("combined_weighted_average_matrix.csv")

Maximum Combined Matrix:

Weighted Average Combined Matrix:


In [24]:
max_combined.to_latex("max_combined.tex", index=True, float_format="%.3f")
weighted_combined.to_latex("weighted_combined.tex", index=True, float_format="%.3f")



```
Combiner strategy : Weighted Average
Simalrity Matrix:

```



In [17]:
weighted_combined

,Diabetes_binary,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,HvyAlcoholConsump,Sex,Age,patient_id,name
Patient_identfication_number,0.357085,0.155704,0.165055,0.160939,0.155228,0.151129,0.192494,0.264740,0.192396,0.197935,0.135417,0.224024,0.756986,0.202173
First_Last_Names,0.265757,0.131901,0.194460,0.194130,0.108640,0.246282,0.213424,0.249130,0.164847,0.174941,0.225958,0.185128,0.246648,0.522899
Age,0.190731,0.208394,0.229092,0.183994,0.193739,0.204006,0.196306,0.180287,0.144582,0.145968,0.279367,0.996325,0.222243,0.270259
Patient_gender,0.415842,0.243119,0.215739,0.154215,0.160227,0.222439,0.235174,0.306470,0.245697,0.186873,0.420680,0.303920,0.640711,0.264446
diabetes,0.743738,0.262952,0.231684,0.220782,0.203391,0.252376,0.340516,0.385472,0.278039,0.266464,0.262655,0.170427,0.330687,0.218859
heart_disease_status,0.467360,0.268702,0.287469,0.250474,0.165030,0.296043,0.327299,0.645287,0.300602,0.242106,0.235869,0.221160,0.407700,0.200093
Systolic_BP,0.392486,0.372730,0.322237,0.268288,0.208783,0.222300,0.304008,0.380065,0.301008,0.312886,0.141226,0.135776,0.315460,0.150401
Diaolistic_BP,0.467582,0.295080,0.260151,0.234340,0.220983,0.223538,0.270532,0.372225,0.326913,0.311147,0.233460,0.153902,0.369298,0.209684
bmi,0.200483,0.278142,0.126130,0.059367,0.994790,0.138954,0.134811,0.107133,0.153816,0.103182,0.164212,0.174371,0.120330,0.175898
cholesterol,0.315526,0.253475,0.470907,0.523057,0.131189,0.342783,0.394880,0.372703,0.271709,0.304765,0.221757,0.166865,0.207185,0.166479




```
Combiner strategy : Maximum
Simalrity Matrix:

```



In [18]:
max_combined

,Diabetes_binary,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,HvyAlcoholConsump,Sex,Age,patient_id,name
Patient_identfication_number,0.529000,0.183507,0.206759,0.198231,0.200000,0.235000,0.235000,0.444000,0.263000,0.350000,0.197361,0.313373,0.856643,0.267000
First_Last_Names,0.571000,0.194502,0.271767,0.272217,0.167000,0.308000,0.308000,0.375000,0.250000,0.278000,0.301929,0.236880,0.462000,0.624165
Age,0.238885,0.270657,0.315819,0.258657,0.322898,0.288677,0.276510,0.211812,0.203970,0.214280,0.385279,1.000000,0.280071,0.400000
Patient_gender,0.615000,0.301531,0.300565,0.228025,0.233379,0.295399,0.290290,0.400000,0.293495,0.246788,0.659466,0.340199,0.800000,0.310076
diabetes,0.803563,0.355920,0.354140,0.333637,0.250000,0.343627,0.445194,0.500000,0.333000,0.399440,0.331092,0.250000,0.500000,0.269432
heart_disease_status,0.615000,0.383170,0.415781,0.348790,0.242717,0.391071,0.392832,0.678145,0.357000,0.309511,0.270448,0.280267,0.500000,0.255488
Systolic_BP,0.475144,0.501884,0.396061,0.336147,0.265638,0.306833,0.398013,0.506442,0.462000,0.375000,0.194043,0.226294,0.401767,0.250668
Diaolistic_BP,0.549303,0.373801,0.308000,0.287567,0.278971,0.317897,0.354553,0.467000,0.429000,0.366579,0.357100,0.223503,0.462000,0.319807
bmi,0.242472,0.333000,0.160551,0.098944,1.000000,0.169591,0.224685,0.148222,0.214026,0.139636,0.273686,0.290618,0.150883,0.210830
cholesterol,0.401877,0.378792,0.535845,0.562095,0.218648,0.400000,0.556000,0.583000,0.332515,0.333000,0.277928,0.232442,0.283975,0.237131


Ground truth Matrix to use it as reference to compute TP TN FN.

compute_quality_metrics is used to compute the quality measures



In [19]:
# Initialize ground truth matrix with 0s
ground_truth = pd.DataFrame([
    # Diabetes_binary, HighBP, HighChol, CholCheck, BMI, Smoker, Stroke, HeartDiseaseorAttack, PhysActivity, HvyAlcoholConsump, Sex, Age, patient_id, name
    [        0,          0,       0,        0,     0,     0,      0,           0,              0,             0,                0,   0,     1,         0],  # Patient_identfication_number
    [        0,          0,       0,        0,     0,     0,      0,           0,              0,             0,                0,   0,     0,         1],  # First_Last_Names
    [        0,          0,       0,        0,     0,     0,      0,           0,              0,             0,                0,   1,     0,         0],  # Age
    [        0,          0,       0,        0,     0,     0,      0,           0,              0,             0,                1,   0,     0,         0],  # Patient_gender
    [        1,          0,       0,        0,     0,     0,      0,           0,              0,             0,                0,   0,     0,         0],  # diabetes
    [        0,          0,       0,        0,     0,     0,      0,           1,              0,             0,                0,   0,     0,         0],  # heart_disease_status
    [        0,          1,       0,        0,     0,     0,      0,           0,              0,             0,                0,   0,     0,         0],  # Systolic_BP (approximated)
    [        0,          0,       0,        0,     0,     0,      0,           0,              0,             0,                0,   0,     0,         0],  # Diaolistic_BP (not present)
    [        0,          0,       0,        0,     1,     0,      0,           0,              0,             0,                0,   0,     0,         0],  # bmi
    [        0,          0,       1,        0,     0,     0,      0,           0,              0,             0,                0,   0,     0,         0],  # cholesterol
    [        0,          0,       0,        0,     0,     1,      0,           0,              0,             0,                0,   0,     0,         0],  # smoker
    [        0,          0,       0,        1,     0,     0,      0,           0,              0,             0,                0,   0,     0,         0],  # medications (CholCheck as proxy)
], index=mediated_schema, columns=dataset_columns)



def compute_quality_metrics(predicted: pd.DataFrame, ground_truth: pd.DataFrame) -> None:
    """Calculate TP, FP, FN, precision, recall, and F1."""
    if predicted.shape != ground_truth.shape:
        raise ValueError("Matrices must be the same shape")

    TP = np.sum((predicted.values == 1) & (ground_truth.values == 1))
    FP = np.sum((predicted.values == 1) & (ground_truth.values == 0))
    FN = np.sum((predicted.values == 0) & (ground_truth.values == 1))

    precision = TP / (TP + FP) if (TP + FP) else 0
    recall = TP / (TP + FN) if (TP + FN) else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) else 0

    print(f"TP: {TP}")
    print(f"FP: {FP}")
    print(f"FN: {FN}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")

In [20]:
def one_to_one_selector(similarity_matrix: pd.DataFrame) -> pd.DataFrame:
    """
    For each row, select the column with the maximum similarity.
    Ensures one-to-one matching by keeping the highest unmatched score across the matrix.
    """
    selected = pd.DataFrame(0, index=similarity_matrix.index, columns=similarity_matrix.columns)
    used_columns = set()

    # Flatten and sort all entries by score descending
    sorted_entries = sorted(
        [(row, col, similarity_matrix.loc[row, col]) for row in similarity_matrix.index for col in similarity_matrix.columns],
        key=lambda x: x[2],
        reverse=True
    )

    for row, col, _ in sorted_entries:
        if row not in selected.index:
            continue
        if selected.loc[row].sum() > 0:
            continue
        if col in used_columns:
            continue
        selected.loc[row, col] = 1
        used_columns.add(col)

    return selected

# similarity_matrix = pd.read_csv("your_combined_similarity_matrix.csv", index_col=0)

max_one_to_one_selected = one_to_one_selector(max_combined)
weighted_one_to_one_selected = one_to_one_selector(weighted_combined)

# Save results
max_one_to_one_selected.to_csv("max_one_to_one_selected.csv")
weighted_one_to_one_selected.to_csv("weighted_one_to_one_selected.csv")

# Evaluate using your ground truth
print("Results for max combinors with one to one selector:")
compute_quality_metrics(max_one_to_one_selected, ground_truth)

print("\n\n\n")

print("Results for weighted average combinors with one to one selector:")
compute_quality_metrics(weighted_one_to_one_selected, ground_truth)

print("\n\n\n")

Results for max combinors with one to one selector:
TP: 9
FP: 3
FN: 2
Precision: 0.7500
Recall: 0.8182
F1 Score: 0.7826




Results for weighted average combinors with one to one selector:
TP: 9
FP: 3
FN: 2
Precision: 0.7500
Recall: 0.8182
F1 Score: 0.7826






In [21]:
weighted_one_to_one_selected

,Diabetes_binary,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,HvyAlcoholConsump,Sex,Age,patient_id,name
Patient_identfication_number,0,0,0,0,0,0,0,0,0,0,0,0,1,0
First_Last_Names,0,0,0,0,0,0,0,0,0,0,0,0,0,1
Age,0,0,0,0,0,0,0,0,0,0,0,1,0,0
Patient_gender,0,0,0,0,0,0,0,0,0,0,1,0,0,0
diabetes,1,0,0,0,0,0,0,0,0,0,0,0,0,0
heart_disease_status,0,0,0,0,0,0,0,1,0,0,0,0,0,0
Systolic_BP,0,1,0,0,0,0,0,0,0,0,0,0,0,0
Diaolistic_BP,0,0,0,0,0,0,0,0,1,0,0,0,0,0
bmi,0,0,0,0,1,0,0,0,0,0,0,0,0,0
cholesterol,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [22]:
max_one_to_one_selected

,Diabetes_binary,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,HvyAlcoholConsump,Sex,Age,patient_id,name
Patient_identfication_number,0,0,0,0,0,0,0,0,0,0,0,0,1,0
First_Last_Names,0,0,0,0,0,0,0,0,0,0,0,0,0,1
Age,0,0,0,0,0,0,0,0,0,0,0,1,0,0
Patient_gender,0,0,0,0,0,0,0,0,0,0,1,0,0,0
diabetes,1,0,0,0,0,0,0,0,0,0,0,0,0,0
heart_disease_status,0,0,0,0,0,0,0,1,0,0,0,0,0,0
Systolic_BP,0,1,0,0,0,0,0,0,0,0,0,0,0,0
Diaolistic_BP,0,0,0,0,0,0,0,0,1,0,0,0,0,0
bmi,0,0,0,0,1,0,0,0,0,0,0,0,0,0
cholesterol,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [25]:
weighted_one_to_one_selected.to_latex("weighted_one_to_one_selected.tex", index=True, float_format="%.3f")
max_one_to_one_selected.to_latex("max_one_to_one_selected.tex", index=True, float_format="%.3f")
ground_truth.to_latex("ground_truth.tex", index=True, float_format="%.3f")